# 전체 시스템 흐름 테스트

이 노트북은 DebateAgent를 사용한 전체 토론 시스템의 흐름을 테스트합니다.

## 테스트 흐름
1. **환경 설정**: 필요한 라이브러리 import 및 설정
2. **데이터 준비**: 데이터셋 빌드 (선택적)
3. **Round 0**: 초기 의견 수집 (각 에이전트의 독립적 예측)
4. **Round 1-N**: 반박(Rebuttal) 및 의견 수정(Revise)
5. **최종 결과**: Ensemble 예측 및 결과 분석


In [ ]:
# 필요한 라이브러리 import
import sys
from pathlib import Path
from core.data_set import build_dataset

# 프로젝트 루트 경로 추가 (노트북 환경 대응)
notebook_dir = Path().resolve()  # 현재 노트북 디렉토리
project_root = notebook_dir.parent  # 프로젝트 루트 (notebooks의 상위 디렉토리)
sys.path.insert(0, str(project_root))

# 테스트 설정
TICKER = "NVDA"  # 테스트할 종목 티커
ROUNDS = 2  # 토론 라운드 수 (1-5 권장)

print(f"테스트 설정 완료")

In [ ]:
from agents.macro_agent import MacroAgent
from agents.sentimental_agent import SentimentalAgent
from agents.technical_agent import TechnicalAgent
from agents.debate_agent import DebateAgent

In [ ]:
debate = DebateAgent(rounds=ROUNDS, ticker=TICKER)

In [ ]:
debate.run()

In [ ]:
debate.opinions

In [ ]:
# build_dataset(TICKER)

In [ ]:
debate.get_opinion(0, TICKER)

In [ ]:
debate.get_rebuttal(1)
debate.get_revise(1)

In [ ]:
debate.get_rebuttal(2)
debate.get_revise(2)

In [ ]:
macro = MacroAgent(ticker=TICKER)
technical = TechnicalAgent(ticker=TICKER)
sentimental = SentimentalAgent(ticker=TICKER)

In [ ]:
data = technical.searcher()

In [ ]:
technical.pretrain()

In [ ]:
technical.predict(data)

In [ ]:
technical.reviewer_draft()

## 1. DebateAgent 초기화

세 가지 에이전트를 포함한 DebateAgent를 초기화합니다:
- **TechnicalAgent**: 기술적 분석가
- **MacroSentiAgent**: 거시경제 분석가  
- **SentimentalAgent**: 센티멘탈 분석가


In [3]:
# 필요한 라이브러리 import
import sys
from pathlib import Path
from core.data_set import build_dataset

# 프로젝트 루트 경로 추가 (노트북 환경 대응)
notebook_dir = Path().resolve()  # 현재 노트북 디렉토리
project_root = notebook_dir.parent  # 프로젝트 루트 (notebooks의 상위 디렉토리)
sys.path.insert(0, str(project_root))

# 테스트 설정
TICKER = "NVDA"  # 테스트할 종목 티커
ROUNDS = 2  # 토론 라운드 수 (1-5 권장)

print(f"테스트 설정 완료")

from agents.debate_agent import DebateAgent

테스트 설정 완료


In [4]:
# DebateAgent 초기화
print("=" * 60)
print("■ DebateAgent 초기화 중...")
print("=" * 60)

debate_agent = DebateAgent(rounds=ROUNDS, ticker=TICKER)

print(f"\n✅ DebateAgent 초기화 완료")
print(f"   - 에이전트 수: {len(debate_agent.agents)}")
print(f"   - 에이전트 목록: {list(debate_agent.agents.keys())}")
print(f"   - 토론 라운드: {debate_agent.rounds}")


■ DebateAgent 초기화 중...

✅ DebateAgent 초기화 완료
   - 에이전트 수: 3
   - 에이전트 목록: ['TechnicalAgent', 'MacroAgent', 'SentimentalAgent']
   - 토론 라운드: 2


## 2. 데이터셋 준비 (선택적)

데이터셋이 이미 존재하는 경우 이 단계를 건너뛸 수 있습니다.


In [ ]:
# 데이터셋 빌드 (필요한 경우에만 실행)
# 주석을 해제하여 실행하세요
print("=" * 60)
print("■ 데이터셋 빌드 중...")
print("=" * 60)
build_dataset(TICKER)
print("✅ 데이터셋 빌드 완료\n")


## 3. Round 0: 초기 의견 수집

각 에이전트가 독립적으로 주식을 분석하고 예측합니다.


In [6]:
# Round 0: 초기 의견 수집
print("=" * 60)
print("■ Round 0: 초기 의견 수집 시작")
print("=" * 60)

opinions_0 = debate_agent.get_opinion(round=0, ticker=TICKER, rebuild=True, force_pretrain=False)

print("\n" + "=" * 60)
print("✅ Round 0 완료: 초기 의견 수집 결과")
print("=" * 60)

for agent_id, opinion in opinions_0.items():
    if opinion and opinion.target:
        print(f"\n[{agent_id}]")
        print(f"  예측 종가: ${opinion.target.next_close:.2f}")
        print(f"  불확실성(σ): {opinion.target.uncertainty:.6f}" if opinion.target.uncertainty else "  불확실성: N/A")
        print(f"  신뢰도(β): {opinion.target.confidence:.4f}" if opinion.target.confidence else "  신뢰도: N/A")
        print(f"  근거 요약: {opinion.reason[:100]}..." if len(opinion.reason) > 100 else f"  근거: {opinion.reason}")


■ Round 0: 초기 의견 수집 시작
[17:29:53] [TechnicalAgent] searcher 실행 (데이터셋 준비)
⚙️ NVDA TechnicalAgent rebuild requested. Building dataset...
[TechnicalAgent] X_seq: (982, 55, 13), y_seq: (982, 1)
✅ NVDA TechnicalAgent dataset saved to CSV (982 samples, 13 features)
[MacroAgent] X_seq: (1196, 40, 13), y_seq: (1196, 1)
✅ NVDA MacroAgent dataset saved to CSV (1196 samples, 13 features)
[SentimentalAgent] X_seq: (1196, 40, 8), y_seq: (1196, 1)
✅ NVDA SentimentalAgent dataset saved to CSV (1196 samples, 8 features)
[17:30:03] [TechnicalAgent] 기존 모델 사용: models\NVDA_TechnicalAgent.pt
[17:30:03] [TechnicalAgent] predict 실행
[17:30:03] [TechnicalAgent] reviewer_draft 실행
  - TechnicalAgent: next_close=178.5078
[17:30:24] [MacroAgent] searcher 실행 (데이터셋 준비)
[INFO] 모델 및 스케일러 로드 중...
model_path: models\NVDA_MacroAgent.pt
[OK] 모델 및 스케일러 로드 완료
[INFO] MacroAgent 데이터 수집 중...
[INFO] Collecting macro features (15 tickers)...
[MacroAgent] Macro data: (44, 91)
[MacroAgent] Stock data: (44, 9)
[MacroAgent] Data sha

[*********************100%***********************]  1 of 1 completed

  - MacroAgent: next_close=184.0353
[17:30:39] [SentimentalAgent] run_dataset 실행
[SentimentalAgent.run_dataset] missing(before): ['return_1d', 'hl_range', 'Volume', 'news_count_1d', 'sentiment_mean_1d']
[SentimentalAgent.run_dataset] all FEATURE_COLS present.
[17:30:39] [SentimentalAgent] 기존 모델 사용: models\NVDA_SentimentalAgent.pt
[17:30:39] [SentimentalAgent] predict 실행 (MC Dropout 포함)
[17:30:39] [SentimentalAgent] reviewer_draft 실행


  - SentimentalAgent: next_close=179.3552
[17:30:45] Round 0 의견 수집 완료 (3 agents)

✅ Round 0 완료: 초기 의견 수집 결과

[TechnicalAgent]
  예측 종가: $178.51
  불확실성(σ): 0.004292
  신뢰도(β): 0.9957
  근거 요약: 1) ma_200/2025-09-10 (0.3381×0.0200=0.0068): 200일 이동평균선은 장기적인 가격 추세를 보여주어 안정적인 매수 신호로 해석됩니다. 2) macd...

[MacroAgent]
  예측 종가: $184.04
  불확실성(σ): 0.040667
  신뢰도(β): 0.9617
  근거 요약: 기술적 분석 측면에서 NVDA는 최근 이동평균선이 상승세를 유지하며 거래량도 안정적으로 증가하고 있어 긍정적인 모멘텀을 보이고 있다. 변동성 지표인 VIX는 낮은 수준을 유지하며 ...

[SentimentalAgent]
  예측 종가: $179.36
  불확실성(σ): 0.027888
  신뢰도(β): 0.9732
  근거 요약: 1) 다음 거래일 종가는 현재가 대비 약 1.5% 상승할 것으로 예상되며, 이는 최근 7일간 긍정적 뉴스 비율이 65.0%로 30일간 58.0%보다 높아 단기 긍정적 분위기가 강화...


## 4. Round 1-N: 반박 및 의견 수정

각 라운드마다:
1. **Rebuttal**: 다른 에이전트의 의견에 대한 반박/지지
2. **Revise**: 신뢰도 기반으로 자신의 의견 수정


In [7]:
# Round 1-N: 반박 및 의견 수정
for round_num in range(1, ROUNDS + 1):
    print("\n" + "=" * 60)
    print(f"■ Round {round_num}: 반박 및 의견 수정")
    print("=" * 60)
    
    # 4-1. Rebuttal (반박)
    print(f"\n[Round {round_num}-1] Rebuttal 수행 중...")
    rebuttals = debate_agent.get_rebuttal(round=round_num)
    print(f"✅ Rebuttal 완료: {len(rebuttals)}개의 반박 생성")
    
    # 반박 내용 일부 출력
    for rebuttal in rebuttals[:2]:  # 처음 2개만 출력
        print(f"\n  [{rebuttal.from_agent_id} → {rebuttal.to_agent_id}]")
        print(f"    스탠스: {rebuttal.stance}")
        print(f"    메시지: {rebuttal.message[:150]}...")
    
    # 4-2. Revise (의견 수정)
    print(f"\n[Round {round_num}-2] Revise 수행 중...")
    revised_opinions = debate_agent.get_revise(round=round_num)
    print(f"✅ Revise 완료: {len(revised_opinions)}개 에이전트 의견 수정")
    
    # 수정된 의견 출력
    print(f"\n[Round {round_num}] 수정된 의견:")
    for agent_id, opinion in revised_opinions.items():
        if opinion and opinion.target:
            prev_opinion = debate_agent.opinions[round_num - 1][agent_id]
            prev_price = prev_opinion.target.next_close if prev_opinion and prev_opinion.target else None
            
            change = opinion.target.next_close - prev_price if prev_price else 0
            change_pct = (change / prev_price * 100) if prev_price else 0
            
            print(f"\n  [{agent_id}]")
            print(f"    이전 예측: ${prev_price:.2f}" if prev_price else "    이전 예측: N/A")
            print(f"    수정 예측: ${opinion.target.next_close:.2f}")
            print(f"    변화: ${change:+.2f} ({change_pct:+.2f}%)")
            print(f"    불확실성: {opinion.target.uncertainty:.6f}" if opinion.target.uncertainty else "    불확실성: N/A")
    
    print(f"\n✅ Round {round_num} 토론 완료")



■ Round 1: 반박 및 의견 수정

[Round 1-1] Rebuttal 수행 중...
[17:30:56] [TechnicalAgent] → [MacroAgent] rebuttal 생성 중...
[17:31:11] [TechnicalAgent] → [SentimentalAgent] rebuttal 생성 중...
[17:31:19] [MacroAgent] → [TechnicalAgent] rebuttal 생성 중...
[17:31:26] [MacroAgent] → [SentimentalAgent] rebuttal 생성 중...
[17:31:33] [SentimentalAgent] → [TechnicalAgent] rebuttal 생성 중...
[17:31:38] [SentimentalAgent] → [MacroAgent] rebuttal 생성 중...
[17:31:44] Round 1 rebuttal 완료 (6개)
✅ Rebuttal 완료: 6개의 반박 생성

  [TechnicalAgent → MacroAgent]
    스탠스: REBUT
    메시지: 현재 NVDA의 기술적 상태를 종합하면, 200일 이동평균(ma_200)의 단기 상승과 OBV의 거래량 증가가 안정적인 상승 추세를 시사합니다. 그러나 MACD는 단기적으로 강한 상승 모멘텀을 보이나, 모멘텀 지표인 RSI는 과매수 구간에 근접하지 않아 모멘텀 강도는 ...

  [TechnicalAgent → SentimentalAgent]
    스탠스: SUPPORT
    메시지: 현재 NVDA의 기술적 상태를 종합하면, 200일 이동평균(ma_200)이 단기와 중기 모두 상승세를 유지하며 장기 추세를 견고히 지지하고 있습니다. OBV 지표도 거래량과 가격 상승을 동반하며 매수세 강화를 나타내고 있어 추세 신호가 긍정적입니다. 모멘텀 측면에서는 ...

[Round 1-2] Revise 수행 중...
[17:31:44] [TechnicalAgent] revise 실행 중...
[Technica

[*********************100%***********************]  1 of 1 completed

[MacroAgent] revise 완료 → new_close=181.55, loss=0.0016965288668870926
[17:32:12] [SentimentalAgent] revise 실행 중...
[SentimentalAgent.run_dataset] missing(before): ['return_1d', 'hl_range', 'Volume', 'news_count_1d', 'sentiment_mean_1d']
[SentimentalAgent.run_dataset] all FEATURE_COLS present.



[*********************100%***********************]  1 of 1 completed


[SentimentalAgent] fine-tuning 완료: loss=0.000000
[SentimentalAgent.run_dataset] missing(before): ['return_1d', 'hl_range', 'Volume', 'news_count_1d', 'sentiment_mean_1d']
[SentimentalAgent.run_dataset] all FEATURE_COLS present.
[SentimentalAgent] revise 완료 → new_close=100.25, loss=0.0
[17:32:17] Round 1 revise 완료 및 opinions 갱신 (3 agents)
✅ Revise 완료: 3개 에이전트 의견 수정

[Round 1] 수정된 의견:

  [TechnicalAgent]
    이전 예측: $178.51
    수정 예측: $178.50
    변화: $-0.01 (-0.00%)
    불확실성: 0.003172

  [MacroAgent]
    이전 예측: $184.04
    수정 예측: $181.55
    변화: $-2.48 (-1.35%)
    불확실성: 0.040667

  [SentimentalAgent]
    이전 예측: $179.36
    수정 예측: $179.36
    변화: $+0.00 (+0.00%)
    불확실성: 0.027888

✅ Round 1 토론 완료

■ Round 2: 반박 및 의견 수정

[Round 2-1] Rebuttal 수행 중...
[17:32:17] [TechnicalAgent] → [MacroAgent] rebuttal 생성 중...
[17:32:26] [TechnicalAgent] → [SentimentalAgent] rebuttal 생성 중...
[17:32:33] [MacroAgent] → [TechnicalAgent] rebuttal 생성 중...
[17:32:41] [MacroAgent] → [SentimentalAgent] rebuttal 생성 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

[MacroAgent] revise 완료 → new_close=180.17, loss=0.009295076131820679
[17:33:25] [SentimentalAgent] revise 실행 중...
[SentimentalAgent.run_dataset] missing(before): ['return_1d', 'hl_range', 'Volume', 'news_count_1d', 'sentiment_mean_1d']
[SentimentalAgent.run_dataset] all FEATURE_COLS present.
[SentimentalAgent] fine-tuning 완료: loss=0.000000
[SentimentalAgent.run_dataset] missing(before): ['return_1d', 'hl_range', 'Volume', 'news_count_1d', 'sentiment_mean_1d']
[SentimentalAgent.run_dataset] all FEATURE_COLS present.


[SentimentalAgent] revise 완료 → new_close=100.26, loss=0.0
[17:33:31] Round 2 revise 완료 및 opinions 갱신 (3 agents)
✅ Revise 완료: 3개 에이전트 의견 수정

[Round 2] 수정된 의견:

  [TechnicalAgent]
    이전 예측: $178.50
    수정 예측: $178.47
    변화: $-0.03 (-0.02%)
    불확실성: 0.003884

  [MacroAgent]
    이전 예측: $181.55
    수정 예측: $180.17
    변화: $-1.39 (-0.76%)
    불확실성: 0.040667

  [SentimentalAgent]
    이전 예측: $179.36
    수정 예측: $179.36
    변화: $+0.00 (+0.00%)
    불확실성: 0.027888

✅ Round 2 토론 완료


In [ ]:
# Round 1-N: 반박 및 의견 수정
for round_num in range(1, ROUNDS + 1):
    print("\n" + "=" * 60)
    print(f"■ Round {round_num}: 반박 및 의견 수정")
    print("=" * 60)
    
    # 4-1. Rebuttal (반박)
    print(f"\n[Round {round_num}-1] Rebuttal 수행 중...")
    rebuttals = debate_agent.get_rebuttal(round=round_num)
    print(f"✅ Rebuttal 완료: {len(rebuttals)}개의 반박 생성")
    
    # 반박 내용 일부 출력
    for rebuttal in rebuttals[:2]:  # 처음 2개만 출력
        print(f"\n  [{rebuttal.from_agent_id} → {rebuttal.to_agent_id}]")
        print(f"    스탠스: {rebuttal.stance}")
        print(f"    메시지: {rebuttal.message[:150]}...")
    
    # 4-2. Revise (의견 수정)
    print(f"\n[Round {round_num}-2] Revise 수행 중...")
    revised_opinions = debate_agent.get_revise(round=round_num)
    print(f"✅ Revise 완료: {len(revised_opinions)}개 에이전트 의견 수정")
    
    # 수정된 의견 출력
    print(f"\n[Round {round_num}] 수정된 의견:")
    for agent_id, opinion in revised_opinions.items():
        if opinion and opinion.target:
            prev_opinion = debate_agent.opinions[round_num - 1][agent_id]
            prev_price = prev_opinion.target.next_close if prev_opinion and prev_opinion.target else None
            
            change = opinion.target.next_close - prev_price if prev_price else 0
            change_pct = (change / prev_price * 100) if prev_price else 0
            
            print(f"\n  [{agent_id}]")
            print(f"    이전 예측: ${prev_price:.2f}" if prev_price else "    이전 예측: N/A")
            print(f"    수정 예측: ${opinion.target.next_close:.2f}")
            print(f"    변화: ${change:+.2f} ({change_pct:+.2f}%)")
            print(f"    불확실성: {opinion.target.uncertainty:.6f}" if opinion.target.uncertainty else "    불확실성: N/A")
    
    print(f"\n✅ Round {round_num} 토론 완료")


## 5. 최종 결과: Ensemble 예측

모든 라운드가 완료된 후 최종 예측 결과를 확인합니다.


In [8]:
# 최종 결과: Ensemble 예측
print("\n" + "=" * 60)
print("■ 최종 결과: Ensemble 예측")
print("=" * 60)

ensemble_result = debate_agent.get_ensemble()

print("\n📊 최종 예측 결과:")
print(f"  종목: {ensemble_result['ticker']}")
print(f"  통화: {ensemble_result['currency']}")
print(f"  현재가: ${ensemble_result['last_price']:.2f}" if ensemble_result['last_price'] else "  현재가: N/A")
print(f"\n  에이전트별 예측:")
for key, value in ensemble_result['agents'].items():
    agent_name = key.replace('_next_close', '')
    print(f"    {agent_name}: ${value:.2f}")

print(f"\n  앙상블 예측:")
print(f"    평균: ${ensemble_result['mean_next_close']:.2f}" if ensemble_result['mean_next_close'] else "    평균: N/A")
print(f"    중앙값: ${ensemble_result['median_next_close']:.2f}" if ensemble_result['median_next_close'] else "    중앙값: N/A")

# 현재가와 비교
if ensemble_result['last_price'] and ensemble_result['mean_next_close']:
    current = ensemble_result['last_price']
    predicted = ensemble_result['mean_next_close']
    change = predicted - current
    change_pct = (change / current) * 100
    print(f"\n  예상 수익률:")
    print(f"    절대 변화: ${change:+.2f}")
    print(f"    상대 변화: {change_pct:+.2f}%")



■ 최종 결과: Ensemble 예측

📊 최종 예측 결과:
  종목: NVDA
  통화: USD
  현재가: $178.88

  에이전트별 예측:
    TechnicalAgent: $178.47
    MacroAgent: $180.17
    SentimentalAgent: $179.36

  앙상블 예측:
    평균: $179.33
    중앙값: $179.36

  예상 수익률:
    절대 변화: $+0.45
    상대 변화: +0.25%


## 6. 라운드별 의견 변화 시각화

각 라운드별로 에이전트의 예측이 어떻게 변화했는지 확인합니다.


In [ ]:
# 라운드별 의견 변화 분석
import pandas as pd
import matplotlib.pyplot as plt

print("\n" + "=" * 60)
print("■ 라운드별 의견 변화 분석")
print("=" * 60)

# 라운드별 의견 데이터 수집
rounds_data = []
for round_num in sorted(debate_agent.opinions.keys()):
    opinions = debate_agent.opinions[round_num]
    for agent_id, opinion in opinions.items():
        if opinion and opinion.target:
            rounds_data.append({
                'Round': round_num,
                'Agent': agent_id,
                'Predicted_Price': opinion.target.next_close,
                'Uncertainty': opinion.target.uncertainty or 0,
                'Confidence': opinion.target.confidence or 0
            })

df_rounds = pd.DataFrame(rounds_data)

if not df_rounds.empty:
    # 라운드별 예측가 변화 시각화
    plt.figure(figsize=(12, 6))
    
    for agent_id in df_rounds['Agent'].unique():
        agent_data = df_rounds[df_rounds['Agent'] == agent_id].sort_values('Round')
        plt.plot(agent_data['Round'], agent_data['Predicted_Price'], 
                marker='o', label=agent_id, linewidth=2, markersize=8)
    
    plt.xlabel('Round', fontsize=12)
    plt.ylabel('Predicted Price ($)', fontsize=12)
    plt.title(f'Round별 예측가 변화 ({TICKER})', fontsize=14, fontweight='bold')
    plt.legend(loc='best', fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 테이블로도 출력
    print("\n📋 라운드별 예측가 변화 테이블:")
    pivot_table = df_rounds.pivot_table(
        index='Agent', 
        columns='Round', 
        values='Predicted_Price',
        aggfunc='first'
    )
    print(pivot_table.round(2))
else:
    print("데이터가 없습니다.")


## 7. 간편 실행: run() 메서드 사용

위의 단계를 한 번에 실행하려면 `run()` 메서드를 사용할 수 있습니다.


In [ ]:
# 간편 실행 (주석 해제하여 사용)
# debate_agent_simple = DebateAgent(rounds=ROUNDS, ticker=TICKER)
# debate_agent_simple.run()
# print(debate_agent_simple.get_ensemble())


## 8. 전체 디베이트 실행 및 문제점 진단

`run()` 메서드를 사용하여 전체 디베이트를 한 번에 실행하고 문제점을 진단합니다.


In [ ]:
# 전체 디베이트 실행 및 문제점 진단
import traceback
from datetime import datetime

print("=" * 80)
print("전체 디베이트 실행 시작")
print("=" * 80)
print(f"시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

# 새로운 DebateAgent 인스턴스 생성
debate_full = DebateAgent(rounds=ROUNDS, ticker=TICKER)

try:
    print("1️⃣ run() 메서드 실행 중...")
    print("-" * 80)
    debate_full.run()
    print("\n✅ run() 메서드 완료!")
    
    print("\n2️⃣ 최종 결과 확인...")
    print("-" * 80)
    ensemble = debate_full.get_ensemble()
    print("\n📊 최종 Ensemble 결과:")
    for key, value in ensemble.items():
        if isinstance(value, dict):
            print(f"  {key}:")
            for k, v in value.items():
                print(f"    {k}: {v}")
        else:
            print(f"  {key}: {value}")
            
except Exception as e:
    print(f"\n❌ 오류 발생: {type(e).__name__}: {e}")
    print("\n" + "=" * 80)
    print("스택 트레이스:")
    print("=" * 80)
    traceback.print_exc()
    
    # 현재 상태 확인
    print("\n" + "=" * 80)
    print("현재 상태 확인:")
    print("=" * 80)
    print(f"  opinions keys: {list(debate_full.opinions.keys())}")
    print(f"  rebuttals keys: {list(debate_full.rebuttals.keys())}")
    if debate_full.opinions:
        print(f"  마지막 라운드: {max(debate_full.opinions.keys())}")
        last_round = max(debate_full.opinions.keys())
        print(f"  마지막 라운드 의견 수: {len(debate_full.opinions[last_round])}")


## 완료

전체 토론 시스템 테스트가 완료되었습니다! 🎉

### 주요 결과 요약:
- ✅ 3개 에이전트의 초기 의견 수집 완료
- ✅ {ROUNDS}라운드의 반박 및 의견 수정 완료
- ✅ 최종 앙상블 예측 생성 완료

### 다음 단계:
- 다른 종목으로 테스트해보기
- 라운드 수를 조정하여 수렴 패턴 관찰
- Streamlit 대시보드에서 인터랙티브하게 테스트
